In [4]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.optimize import differential_evolution

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function with optimized parameters
def triangular_mf(x, a, b, c, ki, wi):
    a = (a + ki) - wi
    b = (b + ki) - wi
    c = (c + ki) - wi
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)
# Fuzzy rule evaluation with optimized parameters
def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    ki, wi = params  # or ki = params[0], wi = params[1] if params is an array
    
    # Membership functions for each feature, adjusted based on ki and wi
    Ageyoung = triangular_mf(Age, 0, 30, 40, ki, wi)
    
    Glucoselow = triangular_mf(Glucose, 0, 85, 100, ki, wi)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160, ki, wi)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200, ki, wi)
    
    INSlow = triangular_mf(INS, 0, 100, 200, ki, wi)
    INSmedium = triangular_mf(INS, 100, 200, 400, ki, wi)
    INShigh = triangular_mf(INS, 200, 400, 600, ki, wi)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30, ki, wi)
    BMImedium = triangular_mf(BMI, 20, 30, 40, ki, wi)
    BMIhigh = triangular_mf(BMI, 30, 40, 50, ki, wi)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6, ki, wi)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0, ki, wi)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0, ki, wi)
    
    # Fuzzy rules
    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    DMverylow = max(Glucoselow, INSlow, BMImedium, DPFlow, Ageyoung)

    # Aggregate into a single value for DM
    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    
    return DM


class HybridHHO_DE:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = None

    def find_rabbit_using_DE(self):
        result = differential_evolution(self.fitness_func, [(0, 100)] * self.dim, popsize=self.n_hawks, maxiter=1, tol=1e-6, init=self.hawks)
        self.rabbit = result.x

    def find_rabbit_using_HHO(self):
        self.rabbit = min(self.hawks, key=self.fitness_func)

    def update_hawks(self):
        new_hawks = []
        for hawk in self.hawks:
            new_hawk = self.update_single_hawk(hawk)
            new_hawks.append(new_hawk)
        self.hawks = np.array(new_hawks)
    
    def update_single_hawk(self, hawk):
        E1 = np.random.uniform(0, 1)
        E2 = np.random.uniform(0, 1)

        if E1 < 0.5:
            if E2 < 0.5:
                return self.rabbit - np.random.uniform(0, 1) * abs(self.rabbit - hawk)
            else:
                return self.rabbit + np.random.uniform(0, 1) * abs(self.rabbit - hawk)
        else:
            return np.random.uniform(0, 100, self.dim)

    def optimize(self):
        for iteration in range(self.n_iterations):
            if iteration % 2 == 0:  # DE update
                self.find_rabbit_using_DE()
            else:  # HHO update
                self.find_rabbit_using_HHO()
                self.update_hawks()

# Initialize hybrid optimizer
hybrid_optimizer = HybridHHO_DE(fitness, dim=2, n_hawks=30, n_iterations=100)

# Run Optimization
hybrid_optimizer.optimize()

# Get the best hawk (solution)
best_hawk = min(hybrid_optimizer.hawks, key=fitness)
print("Best Parameters ki and wi: ", best_hawk)


# Test the classifier on the test set using optimized parameters
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")


Best Parameters ki and wi:  [132.21800945  46.49306153]
Accuracy based on optimized parameters: 0.7077922077922078
Sensitivity based on optimized parameters: 0.6909090909090909
Specificity based on optimized parameters: 0.7171717171717171


In [24]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
dataset = dataset[dataset['Glucose'] !=0]
dataset = dataset[dataset['INS'] !=0]
dataset = dataset[dataset['BMI'] !=0]
dataset = dataset[dataset['DPF'] !=0]
dataset = dataset[dataset['Age'] !=0]

dataset.to_csv('clean_dataset.csv')

In [ ]:
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:


# Constants
DIMENSIONS = 2
N_HAWKS = 30
N_ITERATIONS = 100
OPTIMIZER_TYPE = 'Hybrid'  # Could be 'DE', 'HHO', or 'Hybrid'



# Initialize cache for fitness function
fitness_cache = {}

# Define triangular membership function
def triangular_mf(x, a, b, c):
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

# Fitness function
def fitness(params):
    key = tuple(params)
    if key in fitness_cache:
        return fitness_cache[key]
    
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Cache the result
    result = None
    if OPTIMIZER_TYPE == 'DE':
        result = 1 - (acc * 0.5 + sensitivity * 0.5)
    elif OPTIMIZER_TYPE == 'HHO':
        result = 1 - (acc * 0.5 + specificity * 0.5)
    else:  # Hybrid
        result = 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

    fitness_cache[key] = result
    return result

# Fuzzy rule evaluation with optimized parameters
def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    ki, wi = params  # or ki = params[0], wi = params[1] if params is an array

    Ageyoung = triangular_mf(Age, 0, 30, 40)

    Glucoselow = triangular_mf(Glucose, 0, 85, 100)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200)

    INSlow = triangular_mf(INS, 0, 100, 200)
    INSmedium = triangular_mf(INS, 100, 200, 400)
    INShigh = triangular_mf(INS, 200, 400, 600)

    BMIlow = triangular_mf(BMI, 0, 20, 30)
    BMImedium = triangular_mf(BMI, 20, 30, 40)
    BMIhigh = triangular_mf(BMI, 30, 40, 50)

    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0)

    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)

    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    return DM

def fitness(x):
    return np.sum(x**2)  # Replace with your fitness function

class SequentialHybridHHO_DE:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = np.random.uniform(0, 100, self.dim)
        self.rabbit_fitness = self.fitness_func(self.rabbit)
        self.alpha = 0.5
        self.beta = 0.5
        self.gamma = 0.5
        self.archive = []
        self.tabu_list = []

    def adapt_weights(self):
        self.alpha = np.random.rand()
        self.beta = np.random.rand()
        self.gamma = 1 - self.alpha - self.beta

    def find_rabbit_using_DE(self):
        idxs = np.random.choice(self.n_hawks, 3, replace=False)
        a, b, c = self.hawks[idxs]
        mutant = a + 0.5 * (b - c)
        trial = np.where(np.random.rand(self.dim) < 0.7, mutant, self.rabbit)
        trial_fitness = self.fitness_func(trial)
        
        if trial_fitness < self.rabbit_fitness:
            self.rabbit = trial
            self.rabbit_fitness = trial_fitness

    def find_rabbit_using_HHO(self):
        for hawk in self.hawks:
            hawk_fitness = self.fitness_func(hawk)
            if hawk_fitness < self.rabbit_fitness:
                self.rabbit = hawk
                self.rabbit_fitness = hawk_fitness

    def local_search(self):
        neighborhood = self.rabbit + 0.1 * np.random.randn(*self.rabbit.shape)
        fitness_value = self.fitness_func(neighborhood)
        
        if fitness_value < self.rabbit_fitness:
            self.rabbit = neighborhood
            self.rabbit_fitness = fitness_value

    def update_archive(self):
        self.archive.append((self.rabbit.copy(), self.rabbit_fitness))
        self.archive = sorted(self.archive, key=lambda x: x[1])[:10]

    def swap_with_archive(self):
        if len(self.archive) > 0 and np.random.rand() < 0.1:
            idx = np.random.randint(len(self.archive))
            self.rabbit, self.rabbit_fitness = self.archive[idx]

    def mutate(self, hawk):
        return hawk + 0.05 * np.random.randn(*hawk.shape)

    def update_hawks(self):
        for i, hawk in enumerate(self.hawks):
            self.hawks[i] = self.mutate(hawk)

    def optimize(self):
        for _ in range(self.n_iterations):
            self.adapt_weights()
            rabbit_tuple = tuple(self.rabbit)
            
            if rabbit_tuple not in self.tabu_list:
                if np.random.rand() < self.alpha:
                    self.find_rabbit_using_DE()

                if np.random.rand() < self.beta:
                    self.find_rabbit_using_HHO()

                if np.random.rand() < self.gamma:
                    self.local_search()

                self.update_archive()
                self.swap_with_archive()

            self.update_hawks()
            self.tabu_list.append(rabbit_tuple)
            self.tabu_list = self.tabu_list[-10:]

# Initialize hybrid optimizer
#DIMENSIONS = len(params)  # Replace this with the actual length of your params array
sequential_optimizer = SequentialHybridHHO_DE(fitness, dim=DIMENSIONS)
sequential_optimizer.optimize()

# Get the best solution
best_hawk = min(sequential_optimizer.hawks, key=fitness)
print("Best Parameters ki and wi: ", best_hawk)

# Test on test data
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Performance metrics
print('Accuracy:', accuracy_score(y_test, y_pred_test))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print('Sensitivity:', sensitivity)
print('Specificity:', specificity)


Best Parameters ki and wi:  [2.02080583 5.63281583]
Accuracy: 0.34177215189873417
Sensitivity: 1.0
Specificity: 0.0


In [27]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd

# Step 1: Read CSV file into a DataFrame
df = pd.read_csv('clean_dataset.csv')

# Step 2: Define fuzzy variables
Glucose = ctrl.Antecedent(np.arange(0, 201, 1), 'Glucose')
INS = ctrl.Antecedent(np.arange(0, 1001, 1), 'INS')
BMI = ctrl.Antecedent(np.arange(0, 101, 1), 'BMI')
DPF = ctrl.Antecedent(np.arange(0, 3, 0.01), 'DPF')
Age = ctrl.Antecedent(np.arange(0, 101, 1), 'Age')

DM = ctrl.Consequent(np.arange(0, 6, 1), 'DM')

# Step 3: Membership Functions
Glucose['low'] = fuzz.trimf(Glucose.universe, [0, 50, 100])
Glucose['medium'] = fuzz.trimf(Glucose.universe, [50, 100, 150])
Glucose['high'] = fuzz.trimf(Glucose.universe, [100, 150, 200])

INS['low'] = fuzz.trimf(INS.universe, [0, 250, 500])
INS['medium'] = fuzz.trimf(INS.universe, [250, 500, 750])
INS['high'] = fuzz.trimf(INS.universe, [500, 750, 1000])

BMI['low'] = fuzz.trimf(BMI.universe, [0, 25, 50])
BMI['medium'] = fuzz.trimf(BMI.universe, [25, 50, 75])
BMI['high'] = fuzz.trimf(BMI.universe, [50, 75, 100])

DPF['low'] = fuzz.trimf(DPF.universe, [0, 0.5, 1])
DPF['medium'] = fuzz.trimf(DPF.universe, [0.5, 1, 1.5])
DPF['high'] = fuzz.trimf(DPF.universe, [1, 1.5, 3])

Age['young'] = fuzz.trimf(Age.universe, [0, 20, 40])
Age['middleage'] = fuzz.trimf(Age.universe, [0, 20, 40])
Age['old'] = fuzz.trimf(Age.universe, [0, 20, 40])

DM['verylow'] = fuzz.trimf(DM.universe, [0, 1, 2])
DM['low'] = fuzz.trimf(DM.universe, [1, 2, 3])
DM['medium'] = fuzz.trimf(DM.universe, [2, 3, 4])
DM['high'] = fuzz.trimf(DM.universe, [3, 4, 5])

# Step 4: Rules
# Rule 1
rule1 = ctrl.Rule((Glucose['low'] & INS['low']) | (BMI['low'] & Age['young']), DM['verylow'])

# Rule 2
rule2 = ctrl.Rule((Glucose['low'] & BMI['high']) | (DPF['low'] & Age['young']), DM['low'])

# Rule 3
rule3 = ctrl.Rule((Glucose['medium'] & INS['high']) | (BMI['high'] & DPF['medium']), DM['medium'])

# Rule 4
rule4 = ctrl.Rule((Glucose['high'] & INS['medium']) | (BMI['high'] & DPF['high']), DM['high'])

# Rule 5
rule5 = ctrl.Rule((Glucose['low'] & INS['low']) | (BMI['medium'] & DPF['low']), DM['verylow'])

# Control System and Simulation
DM_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
DM_sim = ctrl.ControlSystemSimulation(DM_ctrl)

# Step 5: Apply fuzzy logic to each row in the dataset
results = []
for _, row in df.iterrows():
    DM_sim.input['Glucose'] = row['Glucose']
    DM_sim.input['INS'] = row['INS']
    DM_sim.input['BMI'] = row['BMI']
    DM_sim.input['DPF'] = row['DPF']
    DM_sim.input['Age'] = row['Age']

    # Compute the fuzzy output
    DM_sim.compute()
    
    # Here I'm assuming that you want to round the output to the nearest integer
    results.append(round(DM_sim.output['DM']))

# Convert results to a NumPy array for easier comparison
results = np.array(results)

# Calculate performance metrics as you did in your original code
acc = accuracy_score(df['Outcome'], results)
tn, fp, fn, tp = confusion_matrix(df['Outcome'], results).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")

ValueError: too many values to unpack (expected 4)